In [1]:
import logging
from filecmp import dircmp
from os import listdir
import os
import filecmp
from dotenv import load_dotenv
from intake import open_catalog
import matplotlib as plt
import pandas as pd
import numpy as np
from shapely import wkt
import shapely
from sqlalchemy import create_engine
from shapely.geometry import shape
from shapely.ops import unary_union
from geopandas import GeoDataFrame
import geopandas as gpd
from datetime import datetime
import networkx as nx
from shapely.geometry import MultiPolygon
import datetime as dt
from pystac_client import Client
from holoviews import opts

load_dotenv()

True

In [2]:
catfeux = open_catalog(f'{os.getenv("PROJECT_PATH")}Fire_Detection_Data_Quality.yaml')

surfdetect_control = catfeux.sentinel_surfaces_detectees_brute.read()
tile_sentinel=catfeux.tile_sentinel2_line_UTM.read()
tile_sentinel=tile_sentinel.to_crs(epsg=4326)

In [3]:
from shapely.geometry import Polygon, mapping
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper

def linestring_to_polygon(gdf):
    polygons = []
    
    for index, row in gdf.iterrows():
        all_coords = mapping(row['geometry'])['coordinates']
        lats = [x[1] for x in all_coords]
        lons = [x[0] for x in all_coords]
        
        polyg = Polygon(zip(lons, lats))
        polygons.append(polyg)
    new_gdf = gpd.GeoDataFrame(geometry=polygons, crs=gdf.crs)
    
    return new_gdf

test = linestring_to_polygon(tile_sentinel)
tile_sentinel['geometry']=test['geometry']
tile_sentinel['Name']='L2A_T'+tile_sentinel['Name']

## Find detection type : "Mono detection" or "pluri detection"

In [5]:
def find_intersecting_id(row, gdf):

    possible_matches_index = list(gdf.sindex.intersection(row['geometry'].bounds))
    possible_matches = gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.geometry.intersects(row['geometry'])]
    intersecting_ids = precise_matches['surface_id'].tolist()
    intersecting_ids = [id_ for id_ in intersecting_ids if id_ != row['surface_id']]
    return intersecting_ids

In [6]:
def mesure_totale(df):
    tot_nb=len(df) #number total of detection 
    tot_surf=df.dissolve() ## total number of detected area (ha)
    tot_surf=pd.Series(tot_surf.area/10000)
    tot_surf=tot_surf.reset_index(drop=True)
    tot_surf=tot_surf[0]
    
    return(tot_surf,tot_nb)

def mesure_pluri_detection(df):
    pluri_detection_list=df[df['groupe_id'].notna()]
    pluri_detection_surface=pluri_detection_list.dissolve() ## number of pluri detected detected area (ha)
    pluri_detection_surface=pd.Series(pluri_detection_surface.area/10000)
    pluri_detection_surface=pluri_detection_surface.reset_index(drop=True)
    pluri_detection_surface=pluri_detection_surface[0]

    pluri_detection_group=pluri_detection_list['groupe_id'].nunique() ## number of group
    pluri_tile_number = pd.DataFrame(pluri_detection_list["nom"].value_counts())
    
    pluri_tile_surface=pluri_detection_list.dissolve(by='nom')
    pluri_tile_surface=pd.DataFrame(pluri_tile_surface.area/10000)
    pluri_tile_surface['nom']=pluri_tile_surface.index
    pluri_tile_surface=pluri_tile_surface.reset_index(drop=True)
    
    return(pluri_tile_surface,pluri_tile_number,pluri_detection_group,pluri_detection_surface)

def mesure_mono_detection(df):
    mono_detection_list=df[df['groupe_id'].isna()]
    mono_detection_surface=mono_detection_list['surface'].sum() ## number of mono detected detected area (ha)
    mono_detection_group=mono_detection_list['groupe_id'].isna().sum() ## number of mono detected polygons

    mono_tile_number = pd.DataFrame(mono_detection_list["nom"].value_counts()) ## number of detection per tiles
    mono_tile_surface = mono_detection_list.groupby('nom')['surface'].sum().reset_index() ## sum of burned area detected per tile
    
    return(mono_tile_surface,mono_tile_number,mono_detection_group,mono_detection_surface)

In [7]:
def try_multiple_date_formats(date_str, formats):
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT 

In [125]:

def stac_search(date_start,date_end,choix):
    from pystac_client import Client

    catalog = Client.open("https://earth-search.aws.element84.com/v1")
    query = catalog.search(
        collections=["sentinel-2-l2a"],datetime=(date_start).strftime('%Y-%m-%d')+'/'+(date_end).strftime('%Y-%m-%d'), bbox=[163.362, -22.76, 168.223, -19.479],fields={"include": ["properties.grid:code","properties.datetime", "properties.eo:cloud_cover"], "exclude": []}
    )

    items = list(query.items())
    stac_json = query.item_collection_as_dict()

    gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")

    df = gdf.rename(columns={
        'grid:code': 'nom',
        'datetime': 'date_',
        'eo:cloud_cover': 'Cloud_Cover'
    })

    df['nom'] = [x[5:] for x in df['nom']]
    df['nom']='L2A_T'+df['nom'] 

    df=df.reset_index(drop=True)
    date_formats = ['%Y-%m-%dT%H:%M:%SZ', '%Y-%m-%d %H:%M:%S.%f%z','%Y-%m-%dT%H:%M:%S.%fZ']
    df['date_'] = df['date_'].apply(try_multiple_date_formats, formats=date_formats)
    df['date_'] = df['date_'].dt.strftime('%Y-%m-%d')
    df['date_'] = pd.to_datetime(df['date_'])
    df = df.sort_values(by='date_', ascending=True)
    
####### stac images
    small_bbox= tile_sentinel['geometry'][tile_sentinel['Name']==choix].iloc[0]
    bbox = list(small_bbox.bounds)

    crs = "epsg:3857"
    zoom = 2**5  # overview level 5

    xx = stac_load(
        items,
        bands=("red", "green", "blue"),
        crs=crs,
        resolution=10 * zoom,
        chunks={},
        groupby="solar_day",
        bbox=bbox
    )

    xx = xx.compute()
    rgb = xx.isel(time=np.s_[:6])
    rgb_normalized = rgb / 4000.0
    rgb_normalized = rgb_normalized.clip(0, 1)  
    
    return(df,rgb_normalized)

In [ ]:
def apply_formatter(plot, element):
    formatter = DatetimeTickFormatter(
        microseconds=["%Y-%m-%d"],
        milliseconds=["%Y-%m-%d"],
        seconds=["%Y-%m-%d"],
        minsec=["%Y-%m-%d"],
        minutes=["%Y-%m-%d"],
        hourmin=["%Y-%m-%d"],
        hours=["%Y-%m-%d"],
        days=["%Y-%m-%d"],
        months=["%Y-%m-%d"],
        years=["%Y-%m-%d"],
    )
    plot.handles['xaxis'].formatter = formatter

In [70]:
import panel as pn
import numpy as np
from bokeh.models import HoverTool
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure
import datetime as dt
from odc.stac import configure_rio, stac_load
import matplotlib.pyplot as plt
import holoviews as hv
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.models.formatters import DatetimeTickFormatter
from odc.stac import configure_rio, stac_load
import matplotlib
pn.extension()
pn.extension('tabulator')

stylesheet = """
.tabulator-cell {
    font-size: 20px;
}
"""

custom_style = {
    'background': '#f89424',
    'border': '1px solid black',
    'padding': '10px',
    'box-shadow': '5px 5px 5px #bcbcbc'
}
    
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: f89424' if v else '' for v in is_max]

tile_bouton = pn.widgets.RadioButtonGroup(options=['L2A_T58KCC','L2A_T58KCD','L2A_T58KDB','L2A_T58KDC','L2A_T58KEA','L2A_T58KEB','L2A_T58KEC',
            'L2A_T58KFA','L2A_T58KFB','L2A_T58KFC','L2A_T58KGA','L2A_T58KGB','L2A_T58KGC','L2A_T58KGV','L2A_T58KHB'],align='center',stylesheets=[stylesheet],
            button_type='warning',button_style='outline',name='Choose a tile')

### PAGE 1 #########
############ table

def maj_table(date_range,choix):
    hv.extension('bokeh','matplotlib')

    df=surfdetect_control.loc[(surfdetect_control['date_'] >= date_range[0]) & (surfdetect_control['date_'] <= date_range[1])]
    df['groupe_id'] = np.nan

    G = nx.Graph()

    for index, row in df.iterrows():
        intersecting_ids = find_intersecting_id(row, df)
        for id_ in intersecting_ids:
            G.add_edge(row['surface_id'], id_)

    groupes = list(nx.connected_components(G))

    for groupe_id, groupe in enumerate(groupes):
        for id_ in groupe:
            df.loc[df['surface_id'] == id_, 'groupe_id'] = groupe_id

    pluri_tile_surface,pluri_tile_number,pluri_detection_group,pluri_detection_surface=mesure_pluri_detection(df)
    mono_tile_surface,mono_tile_number,mono_detection_group,mono_detection_surface=mesure_mono_detection(df)
    tot_surf,nb_tot=mesure_totale(df)

    info_surfaces = pd.merge(mono_tile_number, mono_tile_surface, on='nom', how='outer')
    info_surfaces = pd.merge(info_surfaces, pluri_tile_number, on='nom', how='outer')
    info_surfaces = pd.merge(info_surfaces, pluri_tile_surface, on='nom', how='outer')

    info_surfaces=info_surfaces.rename(columns={'nom':'Tile name','count_x':'Number of mono detection','surface':'Sum of mono detected area','count_y':'Number of pluri detection',0:'Sum of pluri detected area'})

    table = pn.widgets.Tabulator(info_surfaces, name="Informations à l'échelle des tuiles Sentinel-2",header_align='center', show_index=False,
                stylesheets=[stylesheet])
    table.style.apply(highlight_max)

############################
    date_range = pd.date_range(start=df['date_'].min(), end=df['date_'].max())
    full_date_series = pd.DataFrame(date_range.strftime('%Y-%m-%d'), columns=['date_'])
    full_date_series['date_']=pd.to_datetime(full_date_series['date_'])

    df_tiles = df.groupby(['date_', 'nom']).size().reset_index(name='nombre_occurrences')

    df_tiles = df_tiles[df_tiles['nom'] == choix]
    
    df_tiles = pd.merge(full_date_series, df_tiles, on='date_', how='left')
    df_tiles['nom'] = df_tiles['nom'].fillna(choix)

    df_cloud_cover,stac_image=stac_search(df['date_'].min(),df['date_'].max(),choix)
    df_cloud_cover=df_cloud_cover[df_cloud_cover['nom'] == choix]

    df_cloud_cover = pd.merge(full_date_series, df_cloud_cover, on='date_', how='left')
    df_cloud_cover['nom'] = df_cloud_cover['nom'].fillna(choix)
    df_cloud_cover['Cloud_Cover'] = df_cloud_cover['Cloud_Cover'].fillna(0)

    df_tot= pd.merge(df_tiles, df_cloud_cover, on='date_', how='left')

    x_min = df_tot['date_'].min()
    x_max = df_tot['date_'].max()

    tile = hv.Bars(df_tot,  hv.Dimension('date_'), 'nombre_occurrences').opts(color='orange')
    cc_fig = hv.Area(df_tot, label='Cloud Cover Evolution (%)', kdims=('date_','Date'), vdims=('Cloud_Cover','Cloud Cover (%)')).opts(color='grey')

    tooltips = [
        ('Date', '@date_'),
        ('Cloud Cover', '@Cloud_Cover'),
        ('Occurence', '@nombre_occurrences')
    ]
    hover = HoverTool(tooltips=tooltips)

    combined = (tile + cc_fig).cols(1)
    date_formatter = '%Y-%m-%d' 
    combined.opts(
        opts.Area(height=400, width=1800, xrotation=45,tools=[hover], responsive=True,line_width=1.50,hooks=[apply_formatter],shared_axes=True),
        opts.Bars(height=600, width=1800, xrotation=45,tools=[hover], responsive=True,title="Number of detection per tile and the cloud cover associated",hooks=[apply_formatter],shared_axes=True),
        opts.Layout(shared_axes=True))

    N = 4
    images = []

    # Créer un élément hv.RGB pour chaque image temporelle
    for i in range(len(stac_image['time'])):
        img = stac_image.isel(time=i).to_array(dim="band").sel(band=["red", "green", "blue"]).transpose('y', 'x', 'band')
        rgb_image = hv.RGB(img.values, bounds=(0, 0, img.shape[1], img.shape[0])).opts(title=f"Image {i+1}", width=500, height=500, xaxis=None, yaxis=None)
        images.append(rgb_image)

    # Organiser les images en grille
    layout = hv.Layout(images).opts(width=1200)

    return table,nb_tot,tot_surf,mono_detection_group,pluri_detection_group,mono_detection_surface,pluri_detection_surface,combined,images

total_detection=pn.indicators.Number(name='Totale détection', value=0, format='{value}',colors=[(0,'blue')])
surface_total=pn.indicators.Number(name='Surface totale estimée (ha)', value=0, format='{value}',colors=[(0,'blue')])
mono_detection_group=pn.indicators.Number(name='Nombre de Mono détection', value=0, format='{value}',colors=[(0,'red')])
pluri_detection_group=pn.indicators.Number(name='Nombre de Pluri détections', value=0, format='{value}',colors=[(0,'green')])
mono_detection_surface=pn.indicators.Number(name='Surface (ha) Mono détection', value=0, format='{value}',colors=[(0,'red')])
pluri_detection_surface=pn.indicators.Number(name='Surface (ha) Pluri détections', value=0, format='{value}',colors=[(0,'green')])

def update_interface_with_tile_choice(event):
    tile_choice = event.new 
    update_interface(None, tile_choice)

def update_interface(event):
    choix = event.new
    table,nb_tot, tot_surf,mono_nb, pluri_nb,mono_surf, pluri_surf,fig,image = maj_table(datetime_range_picker.value,tile_bouton.value) 
    mono_detection_group.value = mono_nb
    pluri_detection_group.value = pluri_nb
    
    mono_detection_surface.value = mono_surf
    pluri_detection_surface.value = pluri_surf

    total_detection.value = nb_tot
    surface_total.value = tot_surf

    main.clear()
    main.append(table) 
    main.append(tile_bouton)
    main.append(fig)
    main.append(image)


datetime_range_picker = pn.widgets.DatetimeRangePicker(name='Select your Date Range', start=dt.datetime(2023, 1, 1), end=dt.datetime(2023, 12, 31))
datetime_range_picker.param.watch(update_interface, 'value')

tile_bouton.param.watch(update_interface, 'value')
### Indicateurs ########

sidebar = pn.Column(datetime_range_picker,"# Indicateurs Globaux", total_detection,surface_total,mono_detection_group, mono_detection_surface,pluri_detection_group,pluri_detection_surface)
main = pn.Column("Select un intervalle de date pour voir les données") 

template =pn.template.FastListTemplate(
    site="Panel", header_background ='#f89424',title="Dashboard Contrôle des surfaces brûlées en sortie en chaîne",logo="https://neotech.nc/wp-content/uploads/2023/10/logo_oeil_quadri-254x300.jpeg.webp",sidebar=[sidebar],main=[main])

template.servable()

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-2360804045392] Column
        [0] Markdown(str)
    [nav-2360804041296] Column
        [0] DatetimeRangePicker(as_numpy_datetime64=False, end=datetime.datetime(2023, ..., name='Select your Date Range', start=datetime.datetime(2023, ...)
        [1] Markdown(str)
        [2] Number(colors=[(0, 'red')], name='Nombre de Mono d..., value=0)
        [3] Number(colors=[(0, 'red')], name='Surface (ha) M..., value=0)
        [4] Number(colors=[(0, 'green')], name='Nombre de Pluri d..., value=0)
        [5] Number(colors=[(0, 'green')], name='Surface (ha) P..., value=0)